In [ ]:
!pip install transformers datasets peft accelerate evaluate jiwer peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
  

In [ ]:
import os
import zipfile

with zipfile.ZipFile("/content/archive.zip", "r") as zip_ref:
    zip_ref.extractall(".")

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
!pip install wandb

ERROR: Operation cancelled by user


In [ ]:
import torch
from transformers import VisionEncoderDecoderModel, TrOCRProcessor, TrainingArguments, Trainer,DataCollatorForSeq2Seq
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
import evaluate
from datasets import load_dataset
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
model_name = "microsoft/trocr-small-handwritten"

# TrOCRProcessor combines image processing and tokenization
processor = TrOCRProcessor.from_pretrained(model_name)

# Load VisionEncoderDecoderModel pretrained checkpoint for TrOCR small
model = VisionEncoderDecoderModel.from_pretrained(model_name)
model.to(device)

# Configure model for generation and training
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.vocab_size = model.config.decoder.vocab_size


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
class TrOCRDataset(Dataset):
    def __init__(self, csv_path, processor, root_dir):
        """
        Args:
            csv_path (str): Path to CSV with columns [image_name, label]
            processor (TrOCRProcessor): HuggingFace processor (feature extractor + tokenizer)
            root_dir (str): Path to directory containing image files
        """
        self.data = pd.read_csv(csv_path, usecols=[0, 1])
        self.processor = processor
        self.root_dir = root_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get image path and label
        image_name = self.data.iloc[idx, 0]
        label = str(self.data.iloc[idx, 1])
        image_path = os.path.join(self.root_dir, image_name)

        # Load image and convert to RGB
        image = Image.open(image_path).convert("RGB")

        # Preprocess image (pixel values only), no padding
        pixel_values = self.processor(images=image, return_tensors="pt").pixel_values.squeeze()

        # Tokenize label (no padding here)
        labels = self.processor.tokenizer(label, return_tensors="pt").input_ids.squeeze()

        return {
            "pixel_values": pixel_values,  # (3, H, W)
            "labels": labels               # token IDs
        }
class TrOCRDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, batch):
        pixel_values = torch.stack([example["pixel_values"] for example in batch])
        labels = [example["labels"] for example in batch]

        # Pad labels using tokenizer
        batch_labels = self.processor.tokenizer.pad(
            {"input_ids": labels},
            padding=True,
            return_tensors="pt"
        )["input_ids"]

        # Replace pad token ids with -100
        batch_labels[batch_labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values,
            "labels": batch_labels,
        }


NameError: name 'Dataset' is not defined

In [ ]:
train_dataset = TrOCRDataset("/content/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv", processor, "/content/Doctor’s Handwritten Prescription BD dataset/Training/training_words")
val_dataset = TrOCRDataset("/content/Doctor’s Handwritten Prescription BD dataset/Validation/validation_labels.csv", processor, "/content/Doctor’s Handwritten Prescription BD dataset/Validation/validation_words")
test_dataset = TrOCRDataset("/content/Doctor’s Handwritten Prescription BD dataset/Testing/testing_labels.csv", processor, "/content/Doctor’s Handwritten Prescription BD dataset/Testing/testing_words")


NameError: name 'TrOCRDataset' is not defined

In [ ]:
import evaluate

cer_metric = evaluate.load("cer")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode predictions
    decoded_preds = processor.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 with pad token for decoding labels
    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)

    # Clean up text (strip whitespace)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Calculate Character Error Rate (CER)
    cer_metric = evaluate.load("cer")
    cer = cer_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Calculate Word Error Rate (WER)
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Calculate Exact Match Accuracy
    exact_matches = sum(pred == label for pred, label in zip(decoded_preds, decoded_labels))
    accuracy = exact_matches / len(decoded_preds)

    return {
        "cer": cer,
        "wer": wer,
        "accuracy": accuracy
    }


args = training_args = TrainingArguments(
    output_dir="./trocr-finetuned",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=6,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    report_to="none",
    fp16=True,
    push_to_hub=False,
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    remove_unused_columns=True,
    prediction_loss_only=True
)
from transformers import Trainer
data_collator = TrOCRDataCollator(processor)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_loss_func=compute_metrics,
)

# Train
trainer.train()

# Evaluate on test set
results = trainer.evaluate(test_dataset)

# Display all metrics
print("=== Test Results ===")
print(f"Test Loss: {results['eval_loss']:.3f}")

# Debug: see what keys are available
print("Available keys:", list(results.keys()))

# Print available metrics
for key, value in results.items():
    if key.startswith('eval_'):
        print(f"{key}: {value}")
# Performance interpretation
"""cer_percent = results['eval_cer'] * 100
accuracy_percent = results['eval_accuracy'] * 100

print("\n=== Performance Assessment ===")
if cer_percent < 5 and accuracy_percent > 90:
    print(" EXCELLENT performance!")
elif cer_percent < 15 and accuracy_percent > 70:
    print(" GOOD performance!")
elif cer_percent < 25 and accuracy_percent > 50:
    print(" ACCEPTABLE performance - consider more training")
else:
    print(" POOR performance - needs more training or data")

print(f"Character Error Rate: {cer_percent:.1f}% ({'Low' if cer_percent < 10 else 'High'})")
print(f"Exact Match Accuracy: {accuracy_percent:.1f}%")
"""


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.530100,0.767765
2,0.412900,0.383954
3,0.220200,0.275563
4,0.089400,0.262345
5,0.056100,0.219705
6,0.025700,0.218683


=== Test Results ===
Test Loss: 0.533
Available keys: ['eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch']
eval_loss: 0.53336101770401
eval_runtime: 16.6015
eval_samples_per_second: 46.984
eval_steps_per_second: 23.492


'cer_percent = results[\'eval_cer\'] * 100\naccuracy_percent = results[\'eval_accuracy\'] * 100\n\nprint("\n=== Performance Assessment ===")\nif cer_percent < 5 and accuracy_percent > 90:\n    print(" EXCELLENT performance!")\nelif cer_percent < 15 and accuracy_percent > 70:\n    print(" GOOD performance!")\nelif cer_percent < 25 and accuracy_percent > 50:\n    print(" ACCEPTABLE performance - consider more training")\nelse:\n    print(" POOR performance - needs more training or data")\n\nprint(f"Character Error Rate: {cer_percent:.1f}% ({\'Low\' if cer_percent < 10 else \'High\'})")\nprint(f"Exact Match Accuracy: {accuracy_percent:.1f}%")\n'

In [ ]:
print("Available keys:", list(results.keys()))

# Print available metrics
for key, value in results.items():
    if key.startswith('eval_'):
        print(f"{key}: {value}")

Available keys: ['eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch']
eval_loss: 0.5352035164833069
eval_runtime: 16.8584
eval_samples_per_second: 46.268
eval_steps_per_second: 23.134


In [ ]:
# Monitor GPU memory in real-time
import torch
print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"GPU memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
print(f"GPU memory total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Check memory usage during training
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"Memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

GPU memory allocated: 0.23 GB
GPU memory cached: 0.26 GB
GPU memory total: 14.74 GB


In [ ]:
import psutil
memory = psutil.virtual_memory()
print(f"RAM usage: {memory.percent}%")
print(f"Available RAM: {memory.available / 1024**3:.2f} GB")
print(f"Total RAM: {memory.total / 1024**3:.2f} GB")

In [ ]:
# Check if model is actually on GPU
print(f"Model device: {next(model.parameters()).device}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Move model to GPU if not already
model = model.cuda()

Model device: cuda:0
CUDA available: True


In [1]:
import transformers
print(transformers.__version__)

4.53.2
